In [19]:
import os

import django
from django.contrib.auth import get_user_model
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "app.settings")
django.setup()

In [20]:
from api.models import Project, Document, SequenceAnnotation
User = get_user_model()

In [21]:
project = Project.objects.get(id=2)
admin = User.objects.get(username='buriy')
labels = project.labels.all()
docs = project.documents.all()

In [22]:
print(len(docs))
for d in docs:
    print(d.meta, d)

5
{} Орёл / Есть фото в интернете, которые находятся в 
{} ФЗ № 209 , ст 18 п 3 ,хочет получить пояснения по 
{} В начале недели, оставшись одна в закрытой комнате
{} после смерти деда осталось  гладкоствольное огнест
{'uid': 5} Провоз охотничьего оружия в самолете. Из Сургута в


In [23]:
d.meta

"{'uid': 5}"

In [24]:
d

<Document: Провоз охотничьего оружия в самолете. Из Сургута в>

In [25]:
d.is_approved

AttributeError: 'Document' object has no attribute 'is_approved'

In [26]:
d.meta={'uid': 5}

In [27]:
d.save()

In [20]:
def get_doc(d):
    r = []
    for ann in d.seq_annotations.all():
        if d.annotations_approved_by is not None:
            r.append([str(ann.label), ann.start_offset, ann.end_offset])    
    return {'text': d.text, 'annotations': r}

def get_docs(project):
    docs = []
    for d in project.documents.all().filter(annotations_approved_by__isnull=False):
        docs.append(get_doc(d))
    return docs

r = get_doc(d)
print(r)
exported = get_docs(project)
print(exported)


{'text': 'Провоз охотничьего оружия в самолете. Из Сургута в Питер. Какая процедура?', 'annotations': [['Term', 0, 36], ['Term', 64, 73]]}
[{'text': 'Орёл / Есть фото в интернете, которые находятся в общем доступе, может ли клиент скачать и использовать в своей соц. сети или нужно разрешение? Если нужно, как найти? (фотография людей) Если на фотографии, которую скачает клиент, будет подпись фотографа (например в нижнем левом углу), клиент скачает и будет использовать в своей соц. сети, будет ли это нарушением?', 'annotations': [['Term', 132, 142], ['Term', 50, 63], ['Term', 354, 364], ['Term', 236, 253]]}, {'text': 'ФЗ № 209 , ст 18 п 3 ,хочет получить пояснения по данному аспекту . какие организации осуществляют закупку охотничей продукции . они имеют особый оквэд ?', 'annotations': [['Codex', 0, 20], ['Term', 99, 126], ['Term', 146, 151]]}, {'text': 'В начале недели, оставшись одна в закрытой комнате, собака изорвала и изгрызла вещи и одежду моей матери, повредила бытовые приборы. Ст

In [14]:
d.annotations_approved_by
type(d.annotations_approved_by)

django.contrib.auth.models.User

In [21]:
labels = project.labels.all()
label_by_name = {str(l):l for l in labels}

def add_doc(text, meta, labels):
    doc = Document(text=text, meta=meta)
    doc.save()
    for (label, start, end) in labels:
        sa = SequenceAnnotation(start_offset=start, end_offset=end)
        sa.label = label_by_name[labels]
        sa.save()


def existing_docs():
    return set(project.documents.all().values_list('text', flat=True))

# print(existing_docs())

def add_doc_list():
    e = existing_docs()
    
    
    

{'Орёл / Есть фото в интернете, которые находятся в общем доступе, может ли клиент скачать и использовать в своей соц. сети или нужно разрешение? Если нужно, как найти? (фотография людей) Если на фотографии, которую скачает клиент, будет подпись фотографа (например в нижнем левом углу), клиент скачает и будет использовать в своей соц. сети, будет ли это нарушением?', 'после смерти деда осталось  гладкоствольное огнестрельное ружье 16 калибра. Дед имел все необходимые разрешения, но сейчас документы утеряны. Можно ли восстановить документы на оружие и какие разрешительные документы нужно получить клиенту для хранения и ношения оружия.', 'В начале недели, оставшись одна в закрытой комнате, собака изорвала и изгрызла вещи и одежду моей матери, повредила бытовые приборы. Страховки на данные вещи не оформлялось. Дверной замок, естественно, поврежден не был. В целом во всей квартире никаких повреждений нет, кроме вещей в данной комнате.', 'Провоз охотничьего оружия в самолете. Из Сургута в П

In [24]:

len(project.documents.all().filter(annotations_approved_by__isnull=False))

0

In [1]:

text = 'Названы регионы России с\xa0самой высокой смертностью от\xa0рака\nВице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.'

In [8]:
import spacy
nlp = spacy.blank('ru')

In [10]:
doc = nlp.tokenizer(text)

In [25]:
display(f"'{doc[0]} {doc[1]}'",
        doc[0].idx,
        doc[1].idx,
        doc[2].idx,
        text[0:8],
        text[8:16]
)

"'Названы регионы'"

0

8

16

'Названы '

'регионы '

In [17]:
doc

Названы регионы России с самой высокой смертностью от рака
Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.